In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sartajbhuvaji/brain-tumor-classification-mri")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\LENOVO\.cache\kagglehub\datasets\sartajbhuvaji\brain-tumor-classification-mri\versions\3


In [3]:
import os

In [4]:
testing_path = os.path.join(path,os.listdir(path)[0])

In [7]:
training_path = os.path.join(path,os.listdir(path)[1])

In [9]:
os.listdir(training_path)

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [11]:
os.listdir(testing_path)

['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']

In [13]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory

In [15]:
training_dataset = image_dataset_from_directory(training_path,color_mode='grayscale',batch_size=123,image_size=(128,128),
                                                validation_split=0.20,subset='training',seed=123)

Found 2870 files belonging to 4 classes.
Using 2296 files for training.


In [16]:
validation_dataset = image_dataset_from_directory(training_path,color_mode='grayscale',batch_size=123,image_size=(128,128),
                                                validation_split=0.20,subset='validation',seed=123)

Found 2870 files belonging to 4 classes.
Using 574 files for validation.


In [19]:
testing_dataset = image_dataset_from_directory(testing_path,color_mode='grayscale',batch_size=123,image_size=(128,128))

Found 394 files belonging to 4 classes.


In [21]:
model = keras.models.Sequential([
    layers.Input(shape=(128,128,1)),
    layers.Rescaling(1./255),
    layers.Conv2D(128,(3,3),strides=(1,1),activation='relu'),
    layers.MaxPool2D(),
    layers.Conv2D(64,(3,3),strides=(1,1),activation='relu'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(32,activation='relu'),
    layers.Dense(4,activation='softmax')
])

In [23]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [27]:
model.fit(training_dataset,epochs=15,batch_size=123,validation_data=validation_dataset,callbacks=early_stop)

Epoch 1/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.4878 - loss: 1.1548 - val_accuracy: 0.5784 - val_loss: 0.9275
Epoch 2/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 135s 7s/step - accuracy: 0.6424 - loss: 0.8318 - val_accuracy: 0.6951 - val_loss: 0.7501
Epoch 3/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 139s 7s/step - accuracy: 0.7613 - loss: 0.6058 - val_accuracy: 0.7300 - val_loss: 0.6249
Epoch 4/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - accuracy: 0.8301 - loss: 0.4300 - val_accuracy: 0.8118 - val_loss: 0.4739
Epoch 5/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - accuracy: 0.8733 - loss: 0.3334 - val_accuracy: 0.8223 - val_loss: 0.4639
Epoch 6/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 129s 7s/step - accuracy: 0.9233 - loss: 0.2273 - val_accuracy: 0.8310 - val_loss: 0.4298
Epoch 7/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.9551 - loss: 0.1568 - val_accuracy: 0.8258 - val_loss: 0.4431
Epoch 8/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 129s 7s/step - accuracy: 0.9752 - loss: 0.0991 - val_accuracy: 0.8206 - v

In [29]:
model.evaluate(testing_dataset)

4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 879ms/step - accuracy: 0.6853 - loss: 3.1526


[3.152571439743042, 0.6852791905403137]

In [31]:
data_aug = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
])

In [33]:
model = keras.models.Sequential([
    data_aug,
    layers.Rescaling(1./255,input_shape=(128,128,1)),
    layers.Conv2D(128,(3,3),strides=(1,1),activation='relu',padding='same'),
    layers.MaxPool2D(),
    layers.Conv2D(64,(3,3),strides=(1,1),activation='relu',padding='same'),
    layers.MaxPool2D(),
    layers.Conv2D(32,(3,3),strides=(1,1),activation='relu',padding='same'),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(32,activation='relu'),
    layers.Dense(4,activation='softmax')
])

E:\abir\anaconda\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [35]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [37]:
model.fit(training_dataset,epochs=15,batch_size=123,callbacks=early_stop)

Epoch 1/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.4085 - loss: 1.2699
Epoch 2/15


E:\abir\anaconda\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 136s 7s/step - accuracy: 0.5048 - loss: 1.1292
Epoch 3/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 133s 7s/step - accuracy: 0.5588 - loss: 1.0180
Epoch 4/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 131s 7s/step - accuracy: 0.6564 - loss: 0.8373
Epoch 5/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - accuracy: 0.6912 - loss: 0.7582
Epoch 6/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 133s 7s/step - accuracy: 0.7326 - loss: 0.6615
Epoch 7/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - accuracy: 0.7517 - loss: 0.6180
Epoch 8/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 132s 7s/step - accuracy: 0.7718 - loss: 0.5766
Epoch 9/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 134s 7s/step - accuracy: 0.7844 - loss: 0.5480
Epoch 10/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 143s 7s/step - accuracy: 0.8044 - loss: 0.4974
Epoch 11/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 134s 7s/step - accuracy: 0.8149 - loss: 0.4891
Epoch 12/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 926s 51s/step - accuracy: 0.8127 - loss: 0.4492
Epoch 13/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 112s 6s/step - accuracy: 0.8288 -

In [41]:
model.evaluate(testing_dataset)

4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 628ms/step - accuracy: 0.4822 - loss: 2.4754


[2.47542667388916, 0.4822334945201874]

In [43]:
from tensorflow.keras.applications import ResNet50

In [45]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128,128,3))

In [47]:
base_model.trainable = False

In [49]:
training_dataset_2 = image_dataset_from_directory(training_path,color_mode='rgb',batch_size=123,image_size=(128,128),
                                                validation_split=0.20,subset='training',seed=123)

Found 2870 files belonging to 4 classes.
Using 2296 files for training.


In [51]:
validation_dataset_2 = image_dataset_from_directory(training_path,color_mode='rgb',batch_size=123,image_size=(128,128),
                                                validation_split=0.20,subset='validation',seed=123)

Found 2870 files belonging to 4 classes.
Using 574 files for validation.


In [53]:
testing_dataset_2 = image_dataset_from_directory(testing_path,color_mode='rgb',batch_size=123,image_size=(128,128))

Found 394 files belonging to 4 classes.


In [92]:
model_premodel = keras.models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # single-label classification
])


In [94]:
model_premodel.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [39]:
model_premodel.fit(
    training_dataset_2,
    validation_data=validation_dataset_2,
    epochs=15,
    callbacks=early_stop
)

NameError: name 'model_premodel' is not defined

In [110]:
model_premodel.evaluate(testing_dataset_2)

4/4 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - accuracy: 0.7513 - loss: 1.4453


[1.4452996253967285, 0.7512690424919128]

In [55]:
model_premodel_aug = keras.models.Sequential([
    data_aug,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # single-label classification
])

In [57]:
model_premodel_aug.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [59]:
model_premodel_aug.fit(training_dataset_2,validation_data=validation_dataset_2,epochs=15,callbacks=early_stop)

Epoch 1/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 99s 5s/step - accuracy: 0.5710 - loss: 1.1284 - val_accuracy: 0.7317 - val_loss: 0.6192
Epoch 2/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 88s 5s/step - accuracy: 0.7387 - loss: 0.6529 - val_accuracy: 0.7404 - val_loss: 0.6631
Epoch 3/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 0.7779 - loss: 0.5656 - val_accuracy: 0.8031 - val_loss: 0.4961
Epoch 4/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 87s 5s/step - accuracy: 0.7970 - loss: 0.5124 - val_accuracy: 0.8136 - val_loss: 0.4952
Epoch 5/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 88s 5s/step - accuracy: 0.8031 - loss: 0.4805 - val_accuracy: 0.8101 - val_loss: 0.5021


In [61]:
model_premodel_aug.evaluate(testing_dataset_2)

4/4 ━━━━━━━━━━━━━━━━━━━━ 18s 4s/step - accuracy: 0.4594 - loss: 1.5798


[1.579785704612732, 0.4593908488750458]

In [63]:
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

In [65]:
data_aug_2 = data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

In [67]:
model_premodel_aug_2 = keras.models.Sequential([
    data_aug,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # single-label classification
])

In [71]:
model_premodel_aug_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
model_premodel_aug_2.fit(training_dataset_2,validation_data=validation_dataset_2,epochs=15,callbacks=early_stop)

Epoch 1/15
 4/19 ━━━━━━━━━━━━━━━━━━━━ 2:32 10s/step - accuracy: 0.4223 - loss: 1.6899